In [61]:
# Importing Libraries

import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import yfinance as yf

In [62]:
yf.pdr_override()

In [63]:
# Import data from Yahoo

def getData(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix

In [64]:
# Portfolio Performance

def portfolioPerformance(weights, meanReturns, covMatrix, Time):
    returns = np.sum(meanReturns*weights)*Time
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(Time)
    return returns, std

# Adding the Tickers of the stocks from the Austrailian Stock Exchange
    
stockList = ['BHP', 'RIO', 'CBA', 'CSL', 'NAB', 'WBC']
stocks = [stock+'.AX' for stock in stockList]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=1000)

returns, meanReturns, covMatrix = getData(stocks, start=startDate, end=endDate)
returns = returns.dropna()

[*********************100%***********************]  6 of 6 completed


In [65]:
# Giving random weights to the stocks and equating the sum to 100

weights = np.random.random(len(returns.columns))
weights /= np.sum(weights)
returns['portfolio'] = returns.dot(weights)

print(returns)

              BHP.AX    CBA.AX    CSL.AX    NAB.AX    RIO.AX    WBC.AX  \
Date                                                                     
2020-04-27 -0.001310  0.000000  0.026295  0.000000 -0.003094 -0.043705   
2020-04-28 -0.000328 -0.004586 -0.005590 -0.027919 -0.014253  0.017735   
2020-04-29  0.020663  0.041972 -0.014366  0.063316 -0.002915  0.051609   
2020-04-30  0.039524  0.026527 -0.019518  0.041129  0.023389  0.037604   
2020-05-01 -0.077589 -0.061413 -0.034514 -0.048349 -0.056222 -0.057740   
...              ...       ...       ...       ...       ...       ...   
2023-01-13  0.005265  0.012550  0.005579  0.016234  0.009326  0.011064   
2023-01-16  0.001007  0.007981  0.013923  0.011182 -0.000981  0.007155   
2023-01-17 -0.011069  0.000838  0.012686  0.000632 -0.012278  0.000000   
2023-01-18 -0.001221  0.001955  0.007812  0.000000  0.010939 -0.004597   
2023-01-19  0.012225  0.010125  0.008264  0.001263  0.032544  0.005458   

            portfolio  
Date         

In [66]:
# Calculating the VaR
# We will take alpha as 5 for the 5th percentile

def historicalVaR(returns, alpha=5):
    """
    This will read in a pandas dataframe of returns or a pandas series of returns
    And this will output the percentile of the distribution at the given alpha confidence level (5 in this example)
    """
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)


# The aggregate function below is a passed user-defined-function will be passed a Series for evaluation. 
# Bascially we are using this to evalaute every column of the above Data Frame.
    
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalVaR, alpha=alpha)
    else:
        raise TypeError("Expected returns to be dataframe or series")

print(historicalVaR(returns, alpha=5))    

BHP.AX      -0.030608
CBA.AX      -0.020583
CSL.AX      -0.023556
NAB.AX      -0.023055
RIO.AX      -0.028567
WBC.AX      -0.022604
portfolio   -0.020728
dtype: float64


In [67]:
# Calculating the CVaR

def historicalCVaR(returns, alpha=5):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the CVaR
    """
    if isinstance(returns, pd.Series):
        belowVaR = returns <= historicalVaR(returns, alpha=alpha)
        return returns[belowVaR].mean()

# The aggregate function below is a passed user-defined-function will be passed a Series for evaluation. 
# Bascially we are using this to evalaute every column of the above Data Frame.
    
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalCVaR, alpha=alpha)
    else:
        raise TypeError("Expected returns to be dataframe or series")

print(historicalCVaR(returns, alpha=5))  

BHP.AX      -0.046384
CBA.AX      -0.030631
CSL.AX      -0.033697
NAB.AX      -0.030812
RIO.AX      -0.042988
WBC.AX      -0.034536
portfolio   -0.026980
dtype: float64


In [68]:
# 1 days

Time = 1
hVaR = -historicalVaR(returns['portfolio'], alpha=5)*np.sqrt(Time)
hCVaR = -historicalCVaR(returns['portfolio'], alpha=5)*np.sqrt(Time)
pRet, pStd = portfolioPerformance(weights, meanReturns, covMatrix, Time)
InitialInvestment = 100000

# Print the Expected Portfolio Return, VaR and CVaR

print('Expected Portfolio Return        :      ', round(InitialInvestment*pRet,2))
print('Value at Risk 95th percentile    :      ', round(InitialInvestment*hVaR,2))
print('Conditional VaR 95th percentile  :      ', round(InitialInvestment*hCVaR,2))

Expected Portfolio Return        :       88.88
Value at Risk 95th percentile    :       2072.81
Conditional VaR 95th percentile  :       2697.98
